# SARSA 코드 분석

In [10]:
import time
import numpy as np
import tkinter as tk
from PIL import ImageTk, Image

np.random.seed(1)
PhotoImage = ImageTk.PhotoImage
UNIT = 100  # 픽셀 수
WIDTH = 5  # 그리드 월드 가로
HEIGHT = 5  # 그리드 월드 세로


# 자식 클래스 Env에 부모 클래스 Tk를 상속받아 사용
class Env(tk.Tk):
    def __init__(self):
        super(Env, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('MC')
        self.geometry('{0}x{1}'.format(WIDTH * UNIT, HEIGHT * UNIT))
        self.shapes = self.load_images()
        self.canvas = self._build_canvas()
        self.texts = []

    def _build_canvas(self):
        canvas = tk.Canvas(self, bg='white',
                           height=HEIGHT * UNIT,
                           width=WIDTH * UNIT)
        # 그리드 생성 - 세로선 긋기
        for c in range(0, WIDTH * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = c, 0, c, HEIGHT * UNIT
            canvas.create_line(x0, y0, x1, y1)
        # 그리드 생성 - 가로선 긋기
        for r in range(0, HEIGHT * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = 0, r, WIDTH * UNIT, r
            canvas.create_line(x0, y0, x1, y1)

        # 캔버스에 이미지 추가
        self.rectangle = canvas.create_image(50, 50, image=self.shapes[0])
        self.triangle1 = canvas.create_image(250, 150, image=self.shapes[1])
        self.triangle2 = canvas.create_image(150, 250, image=self.shapes[1])
        self.circle = canvas.create_image(250, 250, image=self.shapes[2])

        canvas.pack() # canvas 정렬

        return canvas

    # 이미지 불러오기
    def load_images(self):
        rectangle = PhotoImage(
            Image.open("./img/rectangle.png").resize((65, 65)))
        triangle = PhotoImage(
            Image.open("./img/triangle.png").resize((65, 65)))
        circle = PhotoImage(
            Image.open("./img/circle.png").resize((65, 65)))

        return rectangle, triangle, circle

    def text_value(self, row, col, contents, action, font='Helvetica', size=10,
                   style='normal', anchor="nw"):
        # 그리드 하나의 각 큐함수 표시할 텍스트 위치 설정
        if action == 0: # 상
            origin_x, origin_y = 7, 42
        elif action == 1: # 하
            origin_x, origin_y = 85, 42
        elif action == 2: # 좌
            origin_x, origin_y = 42, 5
        else: # 우
            origin_x, origin_y = 42, 77

        x, y = origin_y + (UNIT * col), origin_x + (UNIT * row)
        font = (font, str(size), style)
        text = self.canvas.create_text(x, y, fill="black", text=contents,
                                       font=font, anchor=anchor)
        return self.texts.append(text) # 빈 리스트인 texts에 각각의 큐함수 값 추가

    # 모든 큐함수를 화면에 표시
    def print_value_all(self, q_table):
        for i in self.texts:
            self.canvas.delete(i)
        self.texts.clear()
        for x in range(HEIGHT):
            for y in range(WIDTH):
                # 그리드 하나 당 상하좌우에 있는 좌표 출력
                for action in range(0, 4):
                    state = [x, y]
                    if str(state) in q_table.keys():
                        # state를 str으로 바꿔서 하는 이유 -> q_table에 key로 들어가기 때문
                        temp = q_table[str(state)][action]
                        self.text_value(y, x, round(temp, 3), action)

    # 픽셀 좌표값으로 되어있는 값을 그리드 행렬 좌표값으로 변환 -> ex) rectangle의 픽셀값 (50,50)을 그리드 행렬 값 (0,0)으로 변환
    def coords_to_state(self, coords):
        x = int((coords[0] - 50) / 100)
        y = int((coords[1] - 50) / 100)
        return [x, y]

    def reset(self):
        self.update()
        time.sleep(0.5)
        # rectangle의 좌표를 x, y로 받기
        x, y = self.canvas.coords(self.rectangle)
        # Canvas.move(canvas_object, x, y) -> 세모 or 원형으로 가면 물체의 좌표에서 각각 50 - x, 50 - y씩 이동시켜줌
        self.canvas.move(self.rectangle, UNIT / 2 - x, UNIT / 2 - y)
        self.render()
        # 초기 상태의 rectangle 그리드 행렬 값 (0,0) return
        return self.coords_to_state(self.canvas.coords(self.rectangle))

    def step(self, action): # rectangle 을 옮긴다
        # 현재 상태의 rectangle 픽셀 좌표값 가져오기
        state = self.canvas.coords(self.rectangle)
        base_action = np.array([0, 0])
        self.render()

        # ex) state = [150,50] 이고 height, width = 3, UNIT = 100인 경우
        # state[1]의 값이 100보다 크지 않으므로 위로 이동 X
        # 아래로 이동할 때의 (HEIGHT - 1) * UNIT을 하는 이유 : 1 * UNIT = 100으로 이동할 수 있는 값을 남겨놓는 것
        if action == 0:  # 상
            if state[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:  # 하
            if state[1] < (HEIGHT - 1) * UNIT: 
                base_action[1] += UNIT
        elif action == 2:  # 좌
            if state[0] > UNIT:
                base_action[0] -= UNIT
        elif action == 3:  # 우
            if state[0] < (WIDTH - 1) * UNIT:
                base_action[0] += UNIT

        # 에이전트 이동
        self.canvas.move(self.rectangle, base_action[0], base_action[1])
        # 에이전트(빨간 네모)를 가장 상위로 배치 (ex) 세모와 같은 상태에 있는 경우, 네모를 보이게 하겠다)
        self.canvas.tag_raise(self.rectangle)
        next_state = self.canvas.coords(self.rectangle)

        # 보상 함수
        if next_state == self.canvas.coords(self.circle):
            reward = 100
            done = True
        elif next_state in [self.canvas.coords(self.triangle1),
                            self.canvas.coords(self.triangle2)]:
            reward = -100
            done = True
        else:
            reward = 0
            done = False

        next_state = self.coords_to_state(next_state)
        return next_state, reward, done

    def render(self):
        time.sleep(0.03)
        self.update()

In [ ]:
import numpy as np
import random
from collections import defaultdict
from environment3 import Env


# SARSA 에이전트 class 선언
class SARSAgent:
    # class 선언 시 __init__은 반드시 생성되어야 함 -> 생성자
    def __init__(self, actions):
        self.actions = actions # [0,1,2,3]
        self.step_size = 0.01 # 알파
        self.discount_factor = 0.9 # 할인율
        self.epsilon = 0.1 # 입실론
        # 0을 초기값으로 가지는 큐함수 테이블 생성
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])

    # <s, a, r, s', a'>의 샘플로부터 큐함수를 업데이트
    def learn(self, state, action, reward, next_state, next_action):
        state, next_state = str(state), str(next_state)
        current_q = self.q_table[state][action] # 시간 t에서의 q함수
        next_state_q = self.q_table[next_state][next_action] # 시간 t+1에서의 q함수
        # SARSA q함수 업데이트 수식 적용
        td = reward + self.discount_factor * next_state_q - current_q
        new_q = current_q + self.step_size * td
        # q함수 업데이트
        self.q_table[state][action] = new_q

    # 입실론 탐욕 정책에 따라서 행동을 반환
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 무작위 행동 반환
            action = np.random.choice(self.actions)
        else:
            # 큐함수에 따른 행동 반환
            state = str(state)
            q_list = self.q_table[state]
            action = arg_max(q_list)
        return action


# 큐함수의 값에 따라 최적의 행동을 반환
def arg_max(q_list):
    max_idx_list = np.argwhere(q_list == np.amax(q_list)) # argwhere() : 특정 데이터의 인덱스 반환
    # 2차원으로 담겨있는 max_idx_list를 1차원 배열로 flatten한 후 리스트로 변환 
    max_idx_list = max_idx_list.flatten().tolist()
    # max가 2개 이상인 경우 random으로 인덱스 뽑기
    return random.choice(max_idx_list)


if __name__ == "__main__":
    env = Env()
    agent = SARSAgent(actions=list(range(env.n_actions)))

    for episode in range(1000):
        # 게임 환경과 상태를 초기화
        state = env.reset()
        # 현재 상태에 대한 행동을 선택
        action = agent.get_action(state)

        while True:
            env.render()

            # 행동을 위한 후 다음상태 보상 에피소드의 종료 여부를 받아옴
            next_state, reward, done = env.step(action)
            # 다음 상태에서의 다음 행동 선택
            next_action = agent.get_action(next_state)
            # <s,a,r,s',a'>로 큐함수를 업데이트
            agent.learn(state, action, reward, next_state, next_action)

            state = next_state
            action = next_action

            # 모든 큐함수를 화면에 표시
            env.print_value_all(agent.q_table)

            if done:
                break